In [1]:
packages = ["graphframes:graphframes:0.8.2-spark3.2-s_2.12"]
from setup import PythonSpark
import networkx as nx
import matplotlib.pyplot as plt


pySpark = PythonSpark(packages=packages)
sc, spark, pyspark = pySpark.get_session()

22/11/20 07:41:48 WARN Utils: Your hostname, suong-VivoBook-ASUSLaptop-X515EA-X515EA resolves to a loopback address: 127.0.1.1; using 192.168.1.14 instead (on interface wlo1)
22/11/20 07:41:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/suong/miniconda3/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/suong/.ivy2/cache
The jars for the packages stored in: /home/suong/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-92aaa32d-f557-4286-be02-39ad3f2c84e2;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 183ms :: artifacts dl 9ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------------

In [2]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, LongType
from graphframes import GraphFrame
from pyspark.sql.functions import desc

schema = StructType([
    StructField("dofM", StringType(), True),
    StructField("dofW", StringType(), True),
    StructField("carrier", StringType(), True),
    StructField("tailnum", StringType(), True),
    StructField("flnum", IntegerType(), True),
    StructField("org_id", LongType(), True),
    StructField("origin", StringType(), True),
    StructField("dest_id", LongType(), True),
    StructField("dest", StringType(), True),
    StructField("crsdeptime", DoubleType(), True),
    StructField("deptime", DoubleType(), True),
    StructField("depdelaymins", DoubleType(), True),
    StructField("crsarrtime", DoubleType(), True),
    StructField("arrtime", DoubleType(), True),
    StructField("arrdelay", DoubleType(), True),
    StructField("crselapsedtime", DoubleType(), True),
    StructField("dist", IntegerType(), True)
])

flightsRDD = spark.read.csv("./rita2014jan.csv", header=False, schema=schema,)
flightsRDD = flightsRDD.withColumn('fltspeed', flightsRDD.dist/(flightsRDD.arrtime - flightsRDD.crsarrtime + flightsRDD.arrdelay + flightsRDD.crselapsedtime))
flightsRDD.limit(5).toPandas()

,dofM,dofW,carrier,tailnum,flnum,org_id,origin,dest_id,dest,crsdeptime,deptime,depdelaymins,crsarrtime,arrtime,arrdelay,crselapsedtime,dist,fltspeed
0,1,3,AA,N338AA,1,12478,JFK,12892,LAX,900.0,914.0,14.0,1225.0,1238.0,13.0,385.0,2475,6.021898
1,2,4,AA,N338AA,1,12478,JFK,12892,LAX,900.0,857.0,0.0,1225.0,1226.0,1.0,385.0,2475,6.395349
2,4,6,AA,N327AA,1,12478,JFK,12892,LAX,900.0,1005.0,65.0,1225.0,1324.0,59.0,385.0,2475,4.558011
3,5,7,AA,N323AA,1,12478,JFK,12892,LAX,900.0,1050.0,110.0,1225.0,1415.0,110.0,385.0,2475,3.613139
4,6,1,AA,N319AA,1,12478,JFK,12892,LAX,900.0,917.0,17.0,1225.0,1217.0,0.0,385.0,2475,6.564987


In [3]:
airports = flightsRDD.select('org_id',"origin").distinct()
airports = airports.withColumnRenamed("org_id", "id")
routes = flightsRDD.select('org_id',"dest_id", "dist").distinct()
routes = routes.withColumnRenamed("org_id", "src").withColumnRenamed("dest_id", "dst")

graph = GraphFrame(airports, routes)

How many airports are there?

In [4]:
numairports = graph.vertices.count()
numairports

301

How many routes are there?

In [5]:
numroutes = graph.edges.count()
numroutes

4090

which routes > 1000 miles distance?

In [6]:
graph.edges.filter("dst > 1000").show()

+-----+-----+----+
|  src|  dst|dist|
+-----+-----+----+
|13303|13487|1501|
|14679|14747|1050|
|10721|12478| 187|
|13204|14747|2554|
|14771|10721|2704|
|10140|12478|1826|
|14107|10721|2300|
|13487|13930| 334|
|12478|14747|2422|
|13930|10423| 978|
|12892|11298|1235|
|14771|10423|1504|
|12478|10721| 187|
|11298|14100|1303|
|15376|13930|1437|
|13830|10666|2681|
|13796|12173|2409|
|10423|14771|1504|
|13303|11298|1121|
|12892|13303|2342|
+-----+-----+----+
only showing top 20 rows



How many incoming flights in for each airport?

In [7]:
tree_outDegree = graph.inDegrees
tree_outDegree.sort(desc("inDegree")).show(5)

+-----+--------+
|   id|inDegree|
+-----+--------+
|10397|     152|
|13930|     145|
|11298|     143|
|11292|     132|
|12266|     107|
+-----+--------+
only showing top 5 rows



How many outcoming flights out for each airport?

In [8]:
tree_outDegree = graph.outDegrees
tree_outDegree.sort(desc("outDegree")).show(5)

+-----+---------+
|   id|outDegree|
+-----+---------+
|10397|      153|
|13930|      146|
|11298|      143|
|11292|      132|
|12266|      108|
+-----+---------+
only showing top 5 rows



In [9]:
# sc.setCheckpointDir('graphframes_cps')
# graph.connectedComponents().show(5)

What are the most important airports according to PageRank?

In [10]:
results = graph.pageRank(resetProbability=0.25, tol=0.01)
results.vertices.sort(desc("pagerank")).show(5)
results.edges.sort(desc("weight")).show(5)

+-----+------+------------------+
|   id|origin|          pagerank|
+-----+------+------------------+
|10397|   ATL|11.576827021614074|
|13930|   ORD|11.397237293161902|
|11298|   DFW|11.234292097162035|
|11292|   DEN| 8.685701563346548|
|13487|   MSP|7.0546713091055056|
+-----+------+------------------+
only showing top 5 rows



+-----+-----+----+------+
|  src|  dst|dist|weight|
+-----+-----+----+------+
|10165|10299|1192|   1.0|
|10141|13487| 257|   1.0|
|10170|10299| 253|   1.0|
|10136|11298| 158|   1.0|
|10208|10397| 143|   1.0|
+-----+-----+----+------+
only showing top 5 rows



How many route i can take from A to B then to C?

In [11]:
graph.find("(a)-[]->(b); (b)-[]->(c); (c)-[]->(b); (b)-[]->(a)").dropDuplicates().filter('a.id == 13930 and c.id == 11292').show()

+------------+------------+------------+
|           a|           b|           c|
+------------+------------+------------+
|{13930, ORD}|{12478, JFK}|{11292, DEN}|
|{13930, ORD}|{13029, LNK}|{11292, DEN}|
|{13930, ORD}|{10529, BDL}|{11292, DEN}|
|{13930, ORD}|{13204, MCO}|{11292, DEN}|
|{13930, ORD}|{13244, MEM}|{11292, DEN}|
|{13930, ORD}|{11618, EWR}|{11292, DEN}|
|{13930, ORD}|{14747, SEA}|{11292, DEN}|
|{13930, ORD}|{11503, EGE}|{11292, DEN}|
|{13930, ORD}|{14570, RNO}|{11292, DEN}|
|{13930, ORD}|{15376, TUS}|{11292, DEN}|
|{13930, ORD}|{14100, PHL}|{11292, DEN}|
|{13930, ORD}|{15370, TUL}|{11292, DEN}|
|{13930, ORD}|{10874, CAK}|{11292, DEN}|
|{13930, ORD}|{14908, SNA}|{11292, DEN}|
|{13930, ORD}|{11423, DSM}|{11292, DEN}|
|{13930, ORD}|{13487, MSP}|{11292, DEN}|
|{13930, ORD}|{14683, SAT}|{11292, DEN}|
|{13930, ORD}|{14771, SFO}|{11292, DEN}|
|{13930, ORD}|{14492, RDU}|{11292, DEN}|
|{13930, ORD}|{12441, JAC}|{11292, DEN}|
+------------+------------+------------+
only showing top

i want to take the shortest trip from A to B

In [12]:
paths = graph.bfs(fromExpr="id = '11140'", toExpr="id = 14730")
paths.show()

+------------+-------------------+------------+-------------------+------------+
|        from|                 e0|          v1|                 e1|          to|
+------------+-------------------+------------+-------------------+------------+
|{11140, CRP}|{11140, 11298, 354}|{11298, DFW}|{11298, 14730, 733}|{14730, SDF}|
|{11140, CRP}|{11140, 12266, 201}|{12266, IAH}|{12266, 14730, 788}|{14730, SDF}|
+------------+-------------------+------------+-------------------+------------+



So, how about "i want to take the fastest trip from A to B"

In [19]:
schema = StructType([
    StructField("dofM", StringType(), True),
    StructField("dofW", StringType(), True),
    StructField("carrier", StringType(), True),
    StructField("tailnum", StringType(), True),
    StructField("flnum", IntegerType(), True),
    StructField("org_id", LongType(), True),
    StructField("origin", StringType(), True),
    StructField("dest_id", LongType(), True),
    StructField("dest", StringType(), True),
    StructField("crsdeptime", DoubleType(), True),
    StructField("deptime", DoubleType(), True),
    StructField("depdelaymins", DoubleType(), True),
    StructField("crsarrtime", DoubleType(), True),
    StructField("arrtime", DoubleType(), True),
    StructField("arrdelay", DoubleType(), True),
    StructField("crselapsedtime", DoubleType(), True),
    StructField("dist", IntegerType(), True)
])

flightsRDD = spark.read.csv("./rita2014jan.csv", header=False, schema=schema,)
flightsRDD = flightsRDD.withColumn('flttime', (flightsRDD.crsarrtime - flightsRDD.crsdeptime))
flightsRDD.limit(5).toPandas()

,dofM,dofW,carrier,tailnum,flnum,org_id,origin,dest_id,dest,crsdeptime,deptime,depdelaymins,crsarrtime,arrtime,arrdelay,crselapsedtime,dist,flttime
0,1,3,AA,N338AA,1,12478,JFK,12892,LAX,900.0,914.0,14.0,1225.0,1238.0,13.0,385.0,2475,325.0
1,2,4,AA,N338AA,1,12478,JFK,12892,LAX,900.0,857.0,0.0,1225.0,1226.0,1.0,385.0,2475,325.0
2,4,6,AA,N327AA,1,12478,JFK,12892,LAX,900.0,1005.0,65.0,1225.0,1324.0,59.0,385.0,2475,325.0
3,5,7,AA,N323AA,1,12478,JFK,12892,LAX,900.0,1050.0,110.0,1225.0,1415.0,110.0,385.0,2475,325.0
4,6,1,AA,N319AA,1,12478,JFK,12892,LAX,900.0,917.0,17.0,1225.0,1217.0,0.0,385.0,2475,325.0


In [20]:
airports = flightsRDD.select('org_id',"origin").distinct()
airports = airports.withColumnRenamed("org_id", "id")
routes = flightsRDD.select('org_id',"dest_id", "flttime").distinct()
routes = routes.withColumnRenamed("org_id", "src").withColumnRenamed("dest_id", "dst")

graph = GraphFrame(airports, routes)

In [21]:
paths = graph.bfs(fromExpr="id = '11140'", toExpr="id = 14730")
paths.show()

+------------+--------------------+------------+--------------------+------------+
|        from|                  e0|          v1|                  e1|          to|
+------------+--------------------+------------+--------------------+------------+
|{11140, CRP}|{11140, 11298, 16...|{11298, DFW}|{11298, 14730, 29...|{14730, SDF}|
|{11140, CRP}|{11140, 11298, 16...|{11298, DFW}|{11298, 14730, 30...|{14730, SDF}|
|{11140, CRP}|{11140, 11298, 12...|{11298, DFW}|{11298, 14730, 29...|{14730, SDF}|
|{11140, CRP}|{11140, 11298, 12...|{11298, DFW}|{11298, 14730, 30...|{14730, SDF}|
|{11140, CRP}|{11140, 11298, 12...|{11298, DFW}|{11298, 14730, 29...|{14730, SDF}|
|{11140, CRP}|{11140, 11298, 12...|{11298, DFW}|{11298, 14730, 30...|{14730, SDF}|
|{11140, CRP}|{11140, 12266, 99.0}|{12266, IAH}|{12266, 14730, 31...|{14730, SDF}|
|{11140, CRP}|{11140, 12266, 99.0}|{12266, IAH}|{12266, 14730, 31...|{14730, SDF}|
|{11140, CRP}|{11140, 12266, 99.0}|{12266, IAH}|{12266, 14730, 35...|{14730, SDF}|
|{11